In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import json
import os
from glob import glob

import pandas as pd
from tqdm import tqdm

from utils.db_util import create_postgres_engine, write_to_db
from db_config import (
    PERSON_META_TABLE,
    PERSON_SUMMARY_TABLE,
    PERSON_SPECIALTY_TABLE,
    PERSON_EXPERIENCE_TABLE,
)

In [4]:
pd.set_option('display.max_colwidth', 150)

In [5]:
ROOT_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/parsed_profiles'
# PARSED_DIR_LIST = [
#     'First_1000_1',
# ]
PARSED_DIR = 'First_1000_1'
PARSED_SUB_DIR_LIST = [
    '523',
    '633',
    '574',
]

In [6]:
db_conn = create_postgres_engine()

In [7]:
def read_json(fpath):
    with open(fpath, 'r') as f:
        content = json.load(f)
    return content

In [8]:
def get_person_id_infos(fpath):
    components = fpath.split('/')
    file_name = components[-1].split('.')[0]
    parent_folder = components[-2]
    batch_folder = components[-3]
    person_id = '_'.join([file_name, parent_folder, batch_folder])    
    
    return person_id, parent_folder, batch_folder

In [9]:
def _extract_keys_from_dict(dict_struct, keys):
    """Get values from a dict of dicts"""
    result = dict_struct.copy()
    
    for key in keys:
        result = result[key]
        
    return result

In [10]:
def extract_person_meta(file_path, parsed_content):
    person_id, parent_folder, batch_folder = get_person_id_infos(file_path)
    
    given_name = _extract_keys_from_dict(parsed_content, ['header', 'name', 'given_name'])
    family_name = _extract_keys_from_dict(parsed_content, ['header', 'name', 'family_name'])
    header_title = _extract_keys_from_dict(parsed_content, ['header', 'title'])
    header_location = _extract_keys_from_dict(parsed_content, ['header', 'location'])
    header_industry = _extract_keys_from_dict(parsed_content, ['header', 'industry'])
    
    result = {
        'person_id': person_id,
        'parent_folder': parent_folder,
        'batch_folder': batch_folder,
        'given_name': given_name,
        'family_name': family_name,
        'header_title': header_title,
        'header_location': header_location,
        'header_industry': header_industry,
    }
    
    return result

def extract_person_summary(file_path, parsed_content):
    person_id, parent_folder, batch_folder = get_person_id_infos(file_path)
    person_summary = parsed_content['summary']
    
    return {
        'person_id': person_id,
        'person_summary': person_summary,
    }

def extract_person_specialty(file_path, parsed_content):
    person_id, parent_folder, batch_folder = get_person_id_infos(file_path)
    person_specialty = parsed_content['specialty']
    
    return {
        'person_id': person_id,
        'person_specialty': person_specialty,
    }

def _is_current_experience(experience_item):
    date_end = _extract_keys_from_dict(experience_item, ['period', 'date_end'])
    return date_end == 'Present'

def extract_person_experience(file_path, parsed_content):
    person_id, parent_folder, batch_folder = get_person_id_infos(file_path)
    
    experience_list = parsed_content['experience']
    
    result = []
    
    for idx, experience_item in enumerate(experience_list):
        
        org_name = experience_item['org_summary']
        org_profile_link = experience_item['company_profile']
        org_detail = experience_item['org_detail']
        
        experience_title = experience_item['title']
        experience_location = experience_item['location']
        experience_description = experience_item['description']
        date_start = _extract_keys_from_dict(experience_item, ['period', 'date_start'])
        date_end = _extract_keys_from_dict(experience_item, ['period', 'date_end'])
        duration = _extract_keys_from_dict(experience_item, ['period', 'duration'])
        is_current = _is_current_experience(experience_item)
        
        result.append({
            'person_id': person_id,
            'experience_id': idx,
            'org_name': org_name,
            'org_profile_link': org_profile_link,
            'org_detail': org_detail,
            'experience_title': experience_title,
            'experience_location': experience_location,
            'experience_description': experience_description,
            'date_start': date_start,
            'date_end': date_end,
            'duration': duration,
            'is_current': is_current,
        })

    
    return result


In [11]:
%%time

for sub_dir in tqdm(PARSED_SUB_DIR_LIST):
    parsed_paths = glob(os.path.join(ROOT_DIR, PARSED_DIR, sub_dir, '*.json'))
    
    person_meta_contents = []
    person_summary_contents = []
    person_specialty_contents = []
    person_experience_contents = []
    
    for fpath in parsed_paths:
        parsed_content = read_json(fpath)
        
        person_meta_contents.append(extract_person_meta(fpath, parsed_content))
        person_summary_contents.append(extract_person_summary(fpath, parsed_content))
        person_specialty_contents.append(extract_person_specialty(fpath, parsed_content))
        person_experience_contents.extend(extract_person_experience(fpath, parsed_content))
        
    person_meta_contents = pd.DataFrame(person_meta_contents)
    person_summary_contents = pd.DataFrame(person_summary_contents)
    person_specialty_contents = pd.DataFrame(person_specialty_contents)
    person_experience_contents = pd.DataFrame(person_experience_contents)
    
    write_to_db(person_meta_contents, 
                db_conn, 
                PERSON_META_TABLE, schema='linkedin', if_exists='append')
    
    write_to_db(person_experience_contents, 
                db_conn, 
                PERSON_EXPERIENCE_TABLE, schema='linkedin', if_exists='append')
    
    write_to_db(person_summary_contents, 
                db_conn, 
                PERSON_SUMMARY_TABLE, schema='linkedin', if_exists='append')
    
    write_to_db(person_specialty_contents, 
                db_conn, 
                PERSON_SPECIALTY_TABLE, schema='linkedin', if_exists='append')

100%|██████████| 3/3 [00:42<00:00, 14.06s/it]

CPU times: user 9.65 s, sys: 454 ms, total: 10.1 s
Wall time: 42.2 s


In [ ]:
person_experience_contents.to_sql()

In [ ]:
person_summary_contents

In [ ]:
person_specialty_contents

In [ ]:
person_experience_contents

In [ ]:
person_experience_contents['date_start'].value_counts()